In [1]:
from qdrant_client import QdrantClient , models



In [2]:
client = QdrantClient(url="http://localhost:6333")

In [4]:
client.create_collection(
    collection_name="qdrant_basics",
    vectors_config = models.VectorParams(size=3,distance=models.Distance.COSINE)
)

True

In [5]:
client.upsert(
    collection_name="qdrant_basics",
    points=[
        models.PointStruct(
            id=1,
            payload={"color":"red"},
            vector=[0.1,0.2,0.3]
        ),
        models.PointStruct(
            id=2,
            payload={"color":"blue"},
            vector=[0.22,0.45,0.55]
        )

            ]
    
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [6]:
client.upsert(
    collection_name="qdrant_basics",
    points = [
        models.PointStruct(
            id=3,
            payload={"color":"red"},
            vector=[0.1,0.22,0.5]
        ),
        models.PointStruct(
            id=4,
            payload={"color":"red"},
            vector=[0.2,0.56,0.33]
        )

    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

# Updating the Payloads

In [7]:
client.set_payload(
    collection_name="qdrant_basics",
    payload={"color":"green",
             "size":"medium"},
    points=[1,4]
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
client.set_payload(
    collection_name = "qdrant_basics",
    payload = {"color":"green","size":"medium"},

    points = models.Filter(
        must = [models.FieldCondition(
                key = "color",
                match = models.MatchValue(value = "red")

        ),
        models.FieldCondition(
            key="size",
            match = models.MatchValue(value="large")
        )
        ]
    )

)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
#deleting the payloads

client.clear_payload(
    collection_name = "qdrant_basics",

    points_selector = [1]

)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
#with filter

client.clear_payload(
    collection_name = "qdrant_basics",
    points_selector = models.Filter(
        must = [
            models.FieldCondition(
                key = "color",
                match = models.MatchValue(value = "red")
            ),
            models.FieldCondition(
                key = "size",
                match = models.MatchValue(value = "small")
            )
        ]
    )
)



UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [11]:
#creating indexes according to the payload values

client.create_payload_index(
    collection_name = "qdrant_basics",
    field_name = "color",
    field_schema= "keyword" #this denotes that color is string type with categorical value
)

UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)

In [3]:
#Inserting ten datas with payloads size and color

client.upsert(
    collection_name = "qdrant_basics",
    points = [
        models.PointStruct(
            id=5,
            payload = {
                "color" : "red",
                "size" : "medium"
            },
            vector = [0.11,0.34,0.33]
        ),
        models.PointStruct(
            id=6,
            payload = {
                "color":"black",
                "size" : "small"
            },
            vector = [0.2,0.35,0.27]
        ),
        models.PointStruct(
            id=7,
            payload = {
                "color":"blue",
                "size" : "large"
            },
            vector = [0.21,0.32,0.29]
        ),
        models.PointStruct(
            id=8,
            payload = {
                "color":"green",
                "size" : "medium"
            },
            vector = [0.32,0.11,0.56]
        ),
        models.PointStruct(
            id=9,
            payload = {
                "color":"red",
                "size" : "small"
            },
            vector = [0.22,0.51,0.01]
        ),
        models.PointStruct(
            id=10,
            payload = {
                "color":"blue",
                "size" : "large"
            },
            vector = [0.2,0.35,0.27]
        ),
        models.PointStruct(
            id=11,
            payload = {
                "color":"red",
                "size" : "large"
            },
            vector = [0.11,0.63,0.92]
        ),
        models.PointStruct(
            id=12,
            payload = {
                "color":"green",
                "size" : "medium"
            },
            vector = [0.23,0.82,0.61]
        )
        
    ]
)

UpdateResult(operation_id=11, status=<UpdateStatus.COMPLETED: 'completed'>)

In [4]:
#setting payloads for points having no size payload

client.set_payload(
    collection_name = "qdrant_basics",
    payload = {"size":"small"},
    points = [2,3]
)

UpdateResult(operation_id=12, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
client.set_payload(
    collection_name= "qdrant_basics",
    payload = {
        "color" : "black",
        "size" : "large"
    },
    points = [1]
)

UpdateResult(operation_id=8, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
client.create_payload_index(
    collection_name="qdrant_basics",
    field_name = "size",
    field_schema = "keyword"
)

UpdateResult(operation_id=10, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
#Facet Counts is like the MySQL count aggregation with where and group by clause in vector database
client.facet(
    collection_name = "qdrant_basics",
    key = "color", #this is like the gorup by clause where the count is done grouped by color key(payload)
    facet_filter= models.Filter(must=[models.FieldCondition(
        key = "size",
        match = models.MatchValue(value="large")
    )])
    #this is like the where clause where the count is 
    #only done on the basis of the points where size = large
    #this is optional field 
)

FacetResponse(hits=[FacetValueHit(value='blue', count=2), FacetValueHit(value='black', count=1), FacetValueHit(value='red', count=1)])

In [5]:
client.facet(
    collection_name = "qdrant_basics",
    key = "color"
)

FacetResponse(hits=[FacetValueHit(value='red', count=4), FacetValueHit(value='blue', count=3), FacetValueHit(value='green', count=3), FacetValueHit(value='black', count=2)])

# Searching


In [13]:
client.query_points(
    collection_name = "qdrant_basics",
    query = [0.1,0.2,0.3]
)

QueryResponse(points=[ScoredPoint(id=1, version=8, score=0.9999998, payload={'color': 'black', 'size': 'large'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=12, score=0.9951674, payload={'color': 'blue', 'size': 'small'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=11, version=11, score=0.9851332, payload={'color': 'red', 'size': 'large'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=12, score=0.98176813, payload={'color': 'red', 'size': 'small'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=5, version=11, score=0.9780219, payload={'color': 'red', 'size': 'medium'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=7, version=11, score=0.9572706, payload={'color': 'blue', 'size': 'large'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=12, version=11, score=0.9439634, payload={'color': 'green', 'size': 'medium'}, vector=None, shard_key=None, order_value=None), Sc

In [14]:
#searching with Filtering, Limits and Params

client.query_points(
    collection_name = "qdrant_basics",
    query = [0.22,0.34,0.45],

    query_filter = models.Filter(
        must = [
            models.FieldCondition(
                key = "color",
                match = models.MatchValue(value = "red")
            )
        ]

    ),
    search_params = models.SearchParams(hnsw_ef = 128,exact = False), #here the search params are the custom parameters for search
    #hnsw = graph search algorithm and the ef paramater determines the number nearest neighbours to expand at once, 
    # exact determines whether or not to search for the point with the exact vector
    #indexed_only = determines whether or not to include the vectors that are indexed
    limit = 4 #how many points to return from the search

)

QueryResponse(points=[ScoredPoint(id=5, version=11, score=0.97903967, payload={'color': 'red', 'size': 'medium'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=11, version=11, score=0.9618025, payload={'color': 'red', 'size': 'large'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=12, score=0.9571761, payload={'color': 'red', 'size': 'small'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=9, version=11, score=0.67289865, payload={'color': 'red', 'size': 'small'}, vector=None, shard_key=None, order_value=None)])

In [15]:
#creation of multiple vectors for a point named vectors

client.create_collection(
    collection_name = "named_vectored_collection",
    vectors_config = {
        "image" : models.VectorParams(size = 3, distance=models.Distance.COSINE),
        "text" : models.VectorParams(size = 4, distance=models.Distance.COSINE)
    }
)

True

In [17]:
#inserting 10 points in the named Vectors

client.upsert(
    collection_name = "named_vectored_collection",
    points = [
        models.PointStruct(
            id=1,
            payload = {"color":"red", "size" : "XXL" },
            vector = {
                "image" :[0.1,0.2,0.3],
                "text" : [0.11,0.21,0.31,0.2]
            },

        )
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [18]:
client.upsert(
    collection_name = "named_vectored_collection",
    points = [
        models.PointStruct(
            id=2,
            vector = {
                "image" :[0.23,0.44,0.33],
                "text" : [0.11,0.2,0.23,0.71]
            }
        ),
        models.PointStruct(
            id=3,
            vector = {
                "image" :[0.21,0.34,0.83],
                "text" : [0.71,0.87,0.91,0.21]
            }
        ),
        models.PointStruct(
            id=4,
            vector = {
                "image" :[0.12,0.73,0.93],
                "text" : [0.01,0.02,0.03,0.11]
            }
        )
    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
client.set_payload(
    collection_name = "named_vectored_collection",
    payload = {
        "color":"black",
        "size":"XL"
    },
    points = [2,3,4]
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
client.create_payload_index(
    collection_name = "named_vectored_collection",
    field_name = "color",
    field_schema = "keyword"
)


UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
client.create_payload_index(
    collection_name = "named_vectored_collection",
    field_name = "size",
    field_schema = "keyword"
)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
#searching in the named vector

client.query_points(
    collection_name = "named_vectored_collection",
    query = [0.2,0.1,0.3],
    using = "image" # this specifies which vector among the text and image to use for the search 
)

QueryResponse(points=[ScoredPoint(id=3, version=2, score=0.9429046, payload={'color': 'black', 'size': 'XL'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=0.9285713, payload={'color': 'red', 'size': 'XXL'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=2, score=0.8473035, payload={'color': 'black', 'size': 'XL'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=4, version=2, score=0.8456209, payload={'color': 'black', 'size': 'XL'}, vector=None, shard_key=None, order_value=None)])